Πρώτα φτιάξαμε ένα csv file με όλα τα συστήματα οργάνων του ανθρώπινου οργανισμού, τα όργανα που περιλαμβάνει το καθένα και τις αντίστοιχιες λειτουργίες.

In [1]:
import pandas as pd

df_organs = pd.read_csv('../data/organ_systems.csv', encoding='ISO8859-7')

In [2]:
df_organs

,ΣΥΣΤΗΜΑΤΑ ΟΡΓΑΝΩΝ,ΟΡΟΙ ΑΝΑΖΗΤΗΣΗΣ
0,σύστημα αισθητήριων οργάνων,"μάτια,αυτιά,γλώσσα,μύτη,όραση,ακοή,όσφρηση"
1,νευρικό σύστημα,"νεύρα,εγκέφαλος,μυελός,μυαλό,σκέψη,μνήμη,νευρικό"
2,σύστημα των ενδοκρινών αδένων,"ενδοκρινείς,υποθάλαμος,υπόφυση,θυρεοειδής,παρα..."
3,πεπτικό σύστημα,"στομάχι,σιελογόνοι,οισοφάγος,συκώτι,πάγκρεας,ή..."
4,κυκλοφορικό σύστημα,"καρδιά,αιμοφόρα,αρτηρίες,φλέβες,τριχοειδή,αίμα..."
5,λεμφικό σύστημα,"αντισώματα,άμυνα,λεμφικό,αμυγδαλές,σπλήνα,θύμος"
6,αναπνευστικό σύστημα,"ρινική,φάρυγγας,λάρυγγας,τραχεία,βρόγχος,πνεύμ..."
7,ουροποιητικό σύστημα,"νεφρά,ουρητήρες,ουροδόχος,ουρήθρα,ούρηση,ούρα,..."
8,ερειστικό σύστημα,"ερειστικό,οστά,σύνδεσμοι,τένοντες,χόνδρος,κόκα..."
9,μυϊκό σύστημα,"μύες,κίνηση,μυϊκό"


In [3]:
df_texts = pd.read_csv('../data/proionta_tis_fisis_votana.csv')

In [4]:
import spacy
import el_core_news_sm
nlp = el_core_news_sm.load()

In [5]:
look_up_list1 = []
for i in range(df_organs.shape[0]):
    look_up_list1 += df_organs.iloc[i]['ΟΡΟΙ ΑΝΑΖΗΤΗΣΗΣ'].split(',')

In [6]:
look_up_list1

['μάτια',
 'αυτιά',
 'γλώσσα',
 'μύτη',
 'όραση',
 'ακοή',
 'όσφρηση',
 'νεύρα',
 'εγκέφαλος',
 'μυελός',
 'μυαλό',
 'σκέψη',
 'μνήμη',
 'νευρικό',
 'ενδοκρινείς',
 'υποθάλαμος',
 'υπόφυση',
 'θυρεοειδής',
 'παραθυροειδής',
 'επινεφρίδια',
 'επίφυση',
 'πάγκρεας',
 'θύμος',
 'ορμόνες',
 'βλεννογόνος',
 'ωοθήκες',
 'στομάχι',
 'σιελογόνοι',
 'οισοφάγος',
 'συκώτι',
 'πάγκρεας',
 'ήπαρ',
 'χοληδόχος',
 'έντερο',
 'χώνεψη',
 'πέψη',
 'πεπτικό',
 'καρδιά',
 'αιμοφόρα',
 'αρτηρίες',
 'φλέβες',
 'τριχοειδή',
 'αίμα',
 'αγγεία',
 'κυκλοφορία',
 'κυκλοφορικό',
 'αντισώματα',
 'άμυνα',
 'λεμφικό',
 'αμυγδαλές',
 'σπλήνα',
 'θύμος',
 'ρινική',
 'φάρυγγας',
 'λάρυγγας',
 'τραχεία',
 'βρόγχος',
 'πνεύμονας',
 'διάφραγμα',
 'αναπνοή',
 'οξυγόνο',
 'αναπνευστικό',
 'νεφρά',
 'ουρητήρες',
 'ουροδόχος',
 'ουρήθρα',
 'ούρηση',
 'ούρα',
 'ουροποιητικό',
 'ερειστικό',
 'οστά',
 'σύνδεσμοι',
 'τένοντες',
 'χόνδρος',
 'κόκαλα',
 'σκελετός',
 'αρθρώσεις',
 'μύες',
 'κίνηση',
 'μυϊκό',
 'καλυπτήριο',
 'δέρμα

In [7]:
organs_look_up_list = []
for search_term in look_up_list1:
    term = nlp(search_term)
    for token in term:
        organs_look_up_list += [token.lemma_]

In [8]:
organs_look_up_list

['μάτι',
 'αυτί',
 'γλώσσα',
 'μύτη',
 'όραση',
 'ακοή',
 'όσφρηση',
 'νεύρα',
 'εγκέφαλος',
 'μυελός',
 'μυαλό',
 'σκέψη',
 'μνήμη',
 'νευρικό',
 'ενδοκρινεί',
 'υποθάλαμος',
 'υπόφυση',
 'θυρεοειδής',
 'παραθυροειδής',
 'επινεφρίδια',
 'επίφυση',
 'πάγκρεας',
 'θύμος',
 'ορμόνα',
 'βλεννογόνος',
 'ωοθήκε',
 'στομάχι',
 'σιελογόνος',
 'οισοφάγος',
 'συκώτι',
 'πάγκρεας',
 'ήπαρ',
 'χοληδόχο',
 'έντερο',
 'χώνεψη',
 'πέψη',
 'πεπτικό',
 'καρδιά',
 'αιμοφόρας',
 'αρτηρία',
 'φλέβα',
 'τριχοειδή',
 'αίμα',
 'αγγείο',
 'κυκλοφορίας',
 'κυκλοφορικό',
 'αντισώματο',
 'άμυνας',
 'λεμφικό',
 'αμυγδαλός',
 'σπλήνα',
 'θύμος',
 'ρινικός',
 'φάρυγγας',
 'λάρυγγας',
 'τραχεία',
 'βρόγχος',
 'πνεύμονας',
 'διάφραγμα',
 'αναπνοή',
 'οξυγόνο',
 'αναπνευστικό',
 'νεφρά',
 'ουρητήρες',
 'ουροδόχο',
 'ουρήθρα',
 'ούρηση',
 'ούρο',
 'ουροποιητικό',
 'ερειστικό',
 'οστά',
 'σύνδεσμοι',
 'τένοντα',
 'χόνδρο',
 'κόκαλας',
 'σκελετός',
 'αρθρώσει',
 'μύα',
 'κίνηση',
 'μυϊκό',
 'καλυπτήριο',
 'δέρμα',
 'μαλ

In [11]:
df_texts['organs'] = ''
import numpy as np
from fuzzywuzzy import process, fuzz

for i in range(df_texts.shape[0]):
    text = df_texts.iloc[i]['text']
    targets = []
    text2 = nlp(text)
    for token in text2:
        for term in organs_look_up_list:
            #if token.lemma_ == term:
            if fuzz.ratio(token.lemma_, term)>90:
                targets += [token.text]
    df_texts.at[i,'organs'] = list(np.unique(targets))

In [12]:
df_texts

,link,text,organs
0,https://www.proionta-tis-fisis.com/10-apo-tis-...,Βότανα 10 από τις εκπληκτικές χρήσεις της α...,"[Έντερο, έντερο, δέρμα, δέρματος, κυκλοφορία, ..."
1,https://www.proionta-tis-fisis.com/10-hriseis-...,Βότανα 10 χρήσεις για την υπέροχη λεβάνταΒό...,"[αυτιά, δέρμα, μαλλιά, μαλλιών, μυαλό, νευρική..."
2,https://www.proionta-tis-fisis.com/10-logi-gia...,Βότανα 10 λόγοι για να φυτέψεις ρίγανη στη ...,[πεπτικό]
3,https://www.proionta-tis-fisis.com/10-votana-p...,Βότανα 10 βότανα που μπορούν να νικούν τις ...,"[Καρυδιά, αναπνευστικού, δέρματος, καρυδιά, κα..."
4,https://www.proionta-tis-fisis.com/15-logoi-na...,Βότανα 15 λόγοι να βάλεις στην ζωή σου το Γ...,"[ήπαρ, αίμα, αγγεία, δέρμα, καρδιάς, λίπος, οσ..."
...,...,...,...
663,https://www.proionta-tis-fisis.com/vrionia-i-k...,Βότανα Βρυωνία η κρητική - ΙδιότητεςΒόταναΒ...,"[δέρμα, μυϊκών, πεπτικού]"
664,https://www.proionta-tis-fisis.com/vrouves-sin...,Βότανα Βρούβες-Σινάπι και διατροφική αξία~Τ...,"[καρδιά, κυκλοφορικό]"
665,https://www.proionta-tis-fisis.com/ydrastis-gi...,Βότανα Υδραστίς για λοιμώξεις του αναπνευστ...,"[αναπνευστικής, αναπνευστικού, αυτί, αυτιών, β..."
666,https://www.proionta-tis-fisis.com/yssopos-o-f...,"Βότανα Ύσσωπος ο φαρμακευτικός, Hyssopus of...","[αναπνευστικού, αναπνοής, καρδιάς, νευρική, νε..."


In [13]:
for i in range(df_texts.shape[0]):
    print(df_texts.iloc[i]['link'],df_texts.iloc[i]['organs'])

https://www.proionta-tis-fisis.com/10-apo-tis-ekpliktikes-hriseis-tis-alois/ ['Έντερο', 'έντερο', 'δέρμα', 'δέρματος', 'κυκλοφορία', 'μαλλιά', 'μαλλιάΗ', 'στομάχι']
https://www.proionta-tis-fisis.com/10-hriseis-gia-tin-iperohi-levanta/ ['αυτιά', 'δέρμα', 'μαλλιά', 'μαλλιών', 'μυαλό', 'νευρική', 'στομάχι']
https://www.proionta-tis-fisis.com/10-logi-gia-na-fitepsis-rigani-sti-glastra/ ['πεπτικό']
https://www.proionta-tis-fisis.com/10-votana-pou-mporoun-na-nikoun-tis-flegmones/ ['Καρυδιά', 'αναπνευστικού', 'δέρματος', 'καρυδιά', 'καρυδιάς', 'κυκλοφορίας', 'μήτρας', 'ουρήθρας', 'πεπτικού', 'πεπτικό', 'στομάχι']
https://www.proionta-tis-fisis.com/15-logoi-na-baleis-stin-zoi-sou-to-gaidouragkatho/ ['ήπαρ', 'αίμα', 'αγγεία', 'δέρμα', 'καρδιάς', 'λίπος', 'οστά', 'πέψη', 'συκώτι', 'συκώτιΗ', 'χοληδόχος']
https://www.proionta-tis-fisis.com/20-fysika-antibiotika-thorakizoun-ton-organismo/ ['άμυνα', 'αίμα', 'αναπνευστικού', 'αναπνευστικών', 'αναπνοή', 'αντισώματα', 'αυτιού', 'βλεννογόνων', 'βρόγχω

https://www.proionta-tis-fisis.com/menta-i-methistiki/ ['δέρμα', 'νευρικών', 'χώνεψη']
https://www.proionta-tis-fisis.com/menta-pathiseis-pou-veltionei-parenergies-allilepidraseis/ ['ήπαρ', 'αίμα', 'δέρμα', 'οισοφάγο', 'συκώτι']
https://www.proionta-tis-fisis.com/midiki-alfalfa-apotoxinonei-to-sikoti-rithmizei-zaharo-gia-petres-sta-nefra/ ['αίμα', 'αρθρώσεις', 'νεφρά', 'ουροδόχου', 'ουροποιητικού', 'πέψη', 'συκώτι']
https://www.proionta-tis-fisis.com/minyanthos-holagogo-votano-gia-therapeia-ton-revmatismon-kai-tis-arthritidas/ ['πέψης', 'χοληδόχου']
https://www.proionta-tis-fisis.com/mithoi-kai-alitheies-gia-ti-stevia/ []
https://www.proionta-tis-fisis.com/moloha-pavi-tas-odinas/ ['ουροδόχου', 'ουροποιητικού', 'πεπτικού']
https://www.proionta-tis-fisis.com/mouria-sykaminos-morea-skamnia-to-dentro-therapeftis/ ['ήπαρ', 'αρθρώσεις', 'αυτιών', 'λίπος', 'μάτια', 'μαλλιών', 'πνεύμονες']
https://www.proionta-tis-fisis.com/mouria-to-farmako-pou-megalonei-stin-avli-mas/ ['έντερο', 'αίμα', 'αίμ

https://www.proionta-tis-fisis.com/to-selino-os-farmakeftiko-fito-hriseis-idiotites/ ['αίμα', 'εγκέφαλο', 'ουροδόχου']
https://www.proionta-tis-fisis.com/to-sinapi-ke-i-therapeftikes-tou-idiotites/ ['έμμηνα', 'καρδιά', 'πεπτικό']
https://www.proionta-tis-fisis.com/to-thymari-apotelesmatikotero-apo-ta-symvatika-antivaktiridiaka-farmaka/ ['αναπνευστικό', 'δέρμα', 'δέρματος', 'κυκλοφορίας', 'πεπτικό']
https://www.proionta-tis-fisis.com/to-tsai-tou-vounou-idiotites-ke-pou-to-vriskoume/ ['αγγεία', 'αγγείων', 'αναπνευστικού', 'αναπνευστικό', 'αναπνοή', 'καρδιάς', 'νευρικού', 'ουροποιητικού', 'πεπτικές']
https://www.proionta-tis-fisis.com/to-votano-pafsipono-ton-samanon/ ['νευρικές']
https://www.proionta-tis-fisis.com/to-votano-pou-prostatevei-to-thyreoeidi-sou/ ['θυρεοειδή']
https://www.proionta-tis-fisis.com/triantafyllia-kai-kynorodo-enas-akros-thetapeftikos-kakpos/ ['άμυνα', 'έντερα', 'δέρμα', 'ουροδόχου', 'ουροποιητικό', 'χοληδόχου']
https://www.proionta-tis-fisis.com/triantafyllo-rodela

In [14]:
df_texts.to_csv('../data/proionta_tis_fisis_votana_entities.csv', index=False)

Στη συνέχεια εξάγουμε μια λίστα γνωστών βοτάνων. Πηγές:[alfavita](https://www.alfavita.gr/epistimi/179616_pliris-katalogos-botanon-kai-oi-therapeies-toys-mathe-tis-astheneies-poy-therapeyei) και όλα τα λεξικά από Α έως Ω στο [proionta-tis-fisis](https://www.proionta-tis-fisis.com/to-lexiko-ton-votanon-ola-apo-a/).

In [1]:
import pandas as pd
df_texts = pd.read_csv('../data/proionta_tis_fisis_votana_entities.csv')

In [2]:
with open('../data/herbs.txt','r',encoding='utf-8') as f:
    content = f.readlines()
content = [x.strip() for x in content] 

In [3]:
for herbs_string in content:
    herbs = herbs_string.split(',')

In [4]:
herbs

['αβρωνία',
 'αγκινάρα',
 'αγιόκλημα',
 'αλθαία',
 'αγγελική',
 'αγγουριά',
 'αγιάγκαθο',
 'κνίκος',
 'αγριάδα',
 'αγριμόνιο',
 'αλόη',
 'αγριοκαστανιά',
 'αδίαντο',
 'άδωνις',
 'άζα',
 'αζαρο',
 'άκανθος',
 'ακόνιτο',
 'άκορος',
 'αλχεμίλη',
 'αμαμελίδα',
 'αμάραντος',
 'άμπελος',
 'αμυγδαλιά',
 'άνηθος',
 'απήγαγος',
 'άρκευθος',
 'άρνικα',
 'ασπερούλα',
 'αστράγαλος',
 'αχιλλέα',
 'αχλαδιά',
 'αψιθιά',
 'αρτεμισία',
 'βάλσαμο',
 'βασιλικός',
 'βατομουριά',
 'βελανιδιά',
 'βέρατρο',
 'βερβένα',
 'βερβερίδα',
 'βερόνικα',
 'βιβούρνο',
 'βρώμη',
 'βρυωνιά',
 'βρωμελαϊνη',
 'βαλεριάνα',
 'σπαθόχορτο',
 'γαρίφαλο',
 'γαρύφαλλο',
 'γέο',
 'γεράνι',
 'γκαλέγκα',
 'γκι',
 'ιξός',
 'γλομπουλάρια',
 'γλιστρίδα',
 'γλυκάνισος',
 'γλυκόριζα',
 'γανόδερμα',
 'γαϊδουράγκαθο',
 'δατούρα',
 'διαβολόχορτο',
 'δάφνη',
 'δαφνοκερασιά',
 'δαχτυλίδα',
 'δελφίνιο',
 'δενδρολίβανο',
 'δίκταμο',
 'δίψακος',
 'δρακοντιά',
 'δυόσμος',
 'εκουιζέτο',
 'πολυκόμπι',
 'ερίγκερο',
 'ερυθραία',
 'ευκάλυπτος',
 'ευώ

In [6]:
import spacy
import el_core_news_sm
nlp = el_core_news_sm.load()

In [7]:
herbs_look_up_list = []
for search_term in herbs:
    term = nlp(search_term)
    for token in term:
        herbs_look_up_list += [token.lemma_]

In [8]:
herbs_look_up_list

['αβρωνίας',
 'αγκινάρα',
 'αγιόκλημα',
 'αλθαία',
 'αγγελικός',
 'αγγουριά',
 'αγιάγκαθο',
 'κνίκο',
 'αγριάδα',
 'αγριμόνιο',
 'αλόη',
 'αγριοκαστανί',
 'αδίαντο',
 'άδωνι',
 'άζο',
 'αζαρο',
 'άκανθος',
 'ακόνιτο',
 'άκορο',
 'αλχεμίλη',
 'αμαμελίδο',
 'αμάραντο',
 'άμπελος',
 'αμυγδαλιά',
 'άνηθο',
 'απήγαγο',
 'άρκευθος',
 'άρνικας',
 'ασπερούλα',
 'αστράγαλος',
 'αχιλλέο',
 'αχλαδιά',
 'αψιθιά',
 'αρτεμισίας',
 'βάλσαμο',
 'βασιλικός',
 'βατομουριά',
 'βελανιδιά',
 'βέρατρο',
 'βερβένα',
 'βερβερίδας',
 'βερόνικας',
 'βιβούρνο',
 'βρώμη',
 'βρυωνί',
 'βρωμελαϊνη',
 'βαλεριάνα',
 'σπαθόχορτο',
 'γαρίφαλο',
 'γαρύφαλλο',
 'γέο',
 'γεράνι',
 'γκαλέγκα',
 'γκι',
 'ιξός',
 'γλομπουλάρι',
 'γλιστρίδα',
 'γλυκάνισο',
 'γλυκόριζα',
 'γανόδερμο',
 'γαϊδουράγκαθο',
 'δατούρα',
 'διαβολόχορτο',
 'δάφνη',
 'δαφνοκερασί',
 'δαχτυλίδας',
 'δελφίνιο',
 'δενδρολίβανο',
 'δίκταμο',
 'δίψακο',
 'δρακοντιά',
 'δυόσμο',
 'εκουιζέτο',
 'πολυκόμπι',
 'ερίγκερος',
 'ερυθραίας',
 'ευκάλυπτος',
 'ευώνυμο

In [9]:
df_texts['herbs'] = ''
import numpy as np
from fuzzywuzzy import process, fuzz

for i in range(df_texts.shape[0]):
    text = df_texts.iloc[i]['text']
    targets = []
    text2 = nlp(text)
    for token in text2:
        for term in herbs_look_up_list:
            #if token.lemma_ == term:
            if fuzz.ratio(token.lemma_, term)>90:
                targets += [token.text]
    df_texts.at[i,'herbs'] = list(np.unique(targets))

In [10]:
df_texts

,link,text,organs,herbs,entities
0,https://www.proionta-tis-fisis.com/10-apo-tis-...,Βότανα 10 από τις εκπληκτικές χρήσεις της α...,"['Έντερο', 'έντερο', 'δέρμα', 'δέρματος', 'κυκ...","[Αλόη, Αλόης, αλόη, αλόης, ελαιόλαδο, μέλι]","['Έντερο', 'έντερο', 'δέρμα', 'δέρματος', 'κυκ..."
1,https://www.proionta-tis-fisis.com/10-hriseis-...,Βότανα 10 χρήσεις για την υπέροχη λεβάνταΒό...,"['αυτιά', 'δέρμα', 'μαλλιά', 'μαλλιών', 'μυαλό...","[θυμάρι, λεβάντα, λεβάντας, μέντα, μαντζουράνα...","['αυτιά', 'δέρμα', 'μαλλιά', 'μαλλιών', 'μυαλό..."
2,https://www.proionta-tis-fisis.com/10-logi-gia...,Βότανα 10 λόγοι για να φυτέψεις ρίγανη στη ...,['πεπτικό'],"[Ρίγανη, δεντρολίβανο, ρίγανη, ρίγανης]","['πεπτικό']['Ρίγανη', 'δεντρολίβανο', 'ρίγανη'..."
3,https://www.proionta-tis-fisis.com/10-votana-p...,Βότανα 10 βότανα που μπορούν να νικούν τις ...,"['Καρυδιά', 'αναπνευστικού', 'δέρματος', 'καρυ...","[Αλόη, Ευκάλυπτος, Καρυδιά, αγριοκαστανιά, αλό...","['Καρυδιά', 'αναπνευστικού', 'δέρματος', 'καρυ..."
4,https://www.proionta-tis-fisis.com/15-logoi-na...,Βότανα 15 λόγοι να βάλεις στην ζωή σου το Γ...,"['ήπαρ', 'αίμα', 'αγγεία', 'δέρμα', 'καρδιάς',...","[Γαϊδουράγκαθο, γαιδουράγκαθο, γαϊδουράγκαθο, ...","['ήπαρ', 'αίμα', 'αγγεία', 'δέρμα', 'καρδιάς',..."
...,...,...,...,...,...
663,https://www.proionta-tis-fisis.com/vrionia-i-k...,Βότανα Βρυωνία η κρητική - ΙδιότητεςΒόταναΒ...,"['δέρμα', 'μυϊκών', 'πεπτικού']","[άμπελο, βρυωνία, κισσού, μανδραγόρα, στάρι]","['δέρμα', 'μυϊκών', 'πεπτικού']['άμπελο', 'βρυ..."
664,https://www.proionta-tis-fisis.com/vrouves-sin...,Βότανα Βρούβες-Σινάπι και διατροφική αξία~Τ...,"['καρδιά', 'κυκλοφορικό']","[Αγγελικής, καρδιά, μέλι, μαρούλι, πράσο, σινά...","['καρδιά', 'κυκλοφορικό']['Αγγελικής', 'καρδιά..."
665,https://www.proionta-tis-fisis.com/ydrastis-gi...,Βότανα Υδραστίς για λοιμώξεις του αναπνευστ...,"['αναπνευστικής', 'αναπνευστικού', 'αυτί', 'αυ...","[Υδραστίς, αγγλική]","['αναπνευστικής', 'αναπνευστικού', 'αυτί', 'αυ..."
666,https://www.proionta-tis-fisis.com/yssopos-o-f...,"Βότανα Ύσσωπος ο φαρμακευτικός, Hyssopus of...","['αναπνευστικού', 'αναπνοής', 'καρδιάς', 'νευρ...","[Άνηθος, Ύσσωπος, καρδιάς]","['αναπνευστικού', 'αναπνοής', 'καρδιάς', 'νευρ..."


In [11]:
df_texts.to_csv('../data/proionta_tis_fisis_votana_entities.csv', index=False)

In [12]:
for i in range(df_texts.shape[0]):
    print(df_texts.iloc[i]['link'],df_texts.iloc[i]['herbs'])

https://www.proionta-tis-fisis.com/10-apo-tis-ekpliktikes-hriseis-tis-alois/ ['Αλόη', 'Αλόης', 'αλόη', 'αλόης', 'ελαιόλαδο', 'μέλι']
https://www.proionta-tis-fisis.com/10-hriseis-gia-tin-iperohi-levanta/ ['θυμάρι', 'λεβάντα', 'λεβάντας', 'μέντα', 'μαντζουράνα', 'τσάι']
https://www.proionta-tis-fisis.com/10-logi-gia-na-fitepsis-rigani-sti-glastra/ ['Ρίγανη', 'δεντρολίβανο', 'ρίγανη', 'ρίγανης']
https://www.proionta-tis-fisis.com/10-votana-pou-mporoun-na-nikoun-tis-flegmones/ ['Αλόη', 'Ευκάλυπτος', 'Καρυδιά', 'αγριοκαστανιά', 'αλόη', 'αλόης', 'βελανιδιάς', 'γλυκάνισου', 'ευκάλυπτος', 'θυμάρι', 'κάρδαμο', 'καρυδιά', 'καρυδιάς', 'λεβάντα', 'λεμονιά', 'μαστίχα', 'μαϊντανό', 'μελιτζάνα', 'πορτοκαλιά', 'σκόρδο']
https://www.proionta-tis-fisis.com/15-logoi-na-baleis-stin-zoi-sou-to-gaidouragkatho/ ['Γαϊδουράγκαθο', 'γαιδουράγκαθο', 'γαϊδουράγκαθο', 'γαϊδουράγκαθου', 'καρδιάς']
https://www.proionta-tis-fisis.com/20-fysika-antibiotika-thorakizoun-ton-organismo/ ['Αλόη', 'Ευκάλυπτος', 'ΕχινάκειαΗ

https://www.proionta-tis-fisis.com/oi-enallaktikes-kai-asynithistes-hriseis-tou-skordou/ ['αβοκάντο', 'βασιλικός', 'μέλι', 'πανάκι', 'σκόρδο', 'σκόρδου']
https://www.proionta-tis-fisis.com/oi-evergetikes-idiotites-tis-louizas-kai-tropoi-na-tin-hrisimopoiisete/ ['λεβάντα', 'λουίζα', 'λουίζας', 'μέλι']
https://www.proionta-tis-fisis.com/oi-exairetikes-idiotites-tis-mantzouranas-kai-pos-hrisimopoieitai/ ['Μαντζουράνα', 'αψιθιά', 'βασιλικός', 'δενδρολίβανο', 'δεντρολίβανο', 'δυόσμο', 'θυμάρι', 'κράταιγος', 'λεβάντα', 'μέλι', 'μέντα', 'μαντζουράνα', 'μαντζουράνας', 'μύρτιλλα', 'ρίγανη', 'ρίγανης', 'τσάι', 'χαμομήλι']
https://www.proionta-tis-fisis.com/oi-panarhaies-therapeftikes-idiotites-tou-safran-krokou/ ['Κρόκος', 'Κρόκου', 'Σαφράν', 'κρόκος', 'σαφράν', 'τζίνσενγκ', 'τσάι']
https://www.proionta-tis-fisis.com/oi-thavmatourges-draseis-tou-votanikou-chaparral/ ['ΓΚΡΑΒΙΌΛΑ', 'Σαπαράλ', 'γαϊδουράγκαθου', 'γκραβιόλα', 'κάνναβης', 'σαπαράλ', 'τσάι']
https://www.proionta-tis-fisis.com/oi-vasike

https://www.proionta-tis-fisis.com/to-ginger-stin-ypiresia-tis-ygeias-mas/ ['Τζίντζερ', 'πιπερόριζα', 'πιπερόριζας', 'τζίντζερ']
https://www.proionta-tis-fisis.com/to-hrisimopoioume-os-diakosmitiko-agnoontas-poso-ofelimo-mporei-na-einai/ ['μέλι']
https://www.proionta-tis-fisis.com/to-iperocho-melissochorto/ ['Μελισσόχορτο', 'βάλσαμο', 'δυόσμο', 'καρδιά', 'λυκίσκο', 'μέντας', 'μελισσόχορτο', 'μελισσόχορτου']
https://www.proionta-tis-fisis.com/to-ishyrotero-antioxeidotiko-pagkosmios-katharizei-to-soma-se-liges-mono-ores-veltionei-tin-ygeia-se-ola-ta-epipeda/ ['καπνό', 'καρδιά', 'καρδιάς', 'πεύκου']
https://www.proionta-tis-fisis.com/to-lexiko-ton-votanon-ola-apo-a/ ['Άμπελος', 'Άνηθος', 'Άρκευθος', 'Αγγελική', 'Αγγουριά', 'Αζαρο', 'Ακόνιτο', 'Αλχεμίλη', 'Αμάραντος', 'Αμαμελίδα', 'Αμυγδαλιά', 'Απήγανος', 'Ασπερούλα', 'Αστράγαλος', 'Αχιλλέα', 'Αχλαδιά', 'Αψιθιά', 'Γλυκόριζα', 'Κερασιά', 'Κράταιγος', 'Κρεμμύδι', 'Μαϊντανός', 'άζα', 'άκορος', 'άκορου', 'άνηθος', 'άρκευθος', 'άρνικα', 'ίνουλα

In [13]:
import pandas as pd
df_texts = pd.read_csv('../data/proionta_tis_fisis_votana_entities.csv')

In [14]:
df_texts

,link,text,organs,herbs,entities
0,https://www.proionta-tis-fisis.com/10-apo-tis-...,Βότανα 10 από τις εκπληκτικές χρήσεις της α...,"['Έντερο', 'έντερο', 'δέρμα', 'δέρματος', 'κυκ...","['Αλόη', 'Αλόης', 'αλόη', 'αλόης', 'ελαιόλαδο'...","['Έντερο', 'έντερο', 'δέρμα', 'δέρματος', 'κυκ..."
1,https://www.proionta-tis-fisis.com/10-hriseis-...,Βότανα 10 χρήσεις για την υπέροχη λεβάνταΒό...,"['αυτιά', 'δέρμα', 'μαλλιά', 'μαλλιών', 'μυαλό...","['θυμάρι', 'λεβάντα', 'λεβάντας', 'μέντα', 'μα...","['αυτιά', 'δέρμα', 'μαλλιά', 'μαλλιών', 'μυαλό..."
2,https://www.proionta-tis-fisis.com/10-logi-gia...,Βότανα 10 λόγοι για να φυτέψεις ρίγανη στη ...,['πεπτικό'],"['Ρίγανη', 'δεντρολίβανο', 'ρίγανη', 'ρίγανης']","['πεπτικό']['Ρίγανη', 'δεντρολίβανο', 'ρίγανη'..."
3,https://www.proionta-tis-fisis.com/10-votana-p...,Βότανα 10 βότανα που μπορούν να νικούν τις ...,"['Καρυδιά', 'αναπνευστικού', 'δέρματος', 'καρυ...","['Αλόη', 'Ευκάλυπτος', 'Καρυδιά', 'αγριοκασταν...","['Καρυδιά', 'αναπνευστικού', 'δέρματος', 'καρυ..."
4,https://www.proionta-tis-fisis.com/15-logoi-na...,Βότανα 15 λόγοι να βάλεις στην ζωή σου το Γ...,"['ήπαρ', 'αίμα', 'αγγεία', 'δέρμα', 'καρδιάς',...","['Γαϊδουράγκαθο', 'γαιδουράγκαθο', 'γαϊδουράγκ...","['ήπαρ', 'αίμα', 'αγγεία', 'δέρμα', 'καρδιάς',..."
...,...,...,...,...,...
663,https://www.proionta-tis-fisis.com/vrionia-i-k...,Βότανα Βρυωνία η κρητική - ΙδιότητεςΒόταναΒ...,"['δέρμα', 'μυϊκών', 'πεπτικού']","['άμπελο', 'βρυωνία', 'κισσού', 'μανδραγόρα', ...","['δέρμα', 'μυϊκών', 'πεπτικού']['άμπελο', 'βρυ..."
664,https://www.proionta-tis-fisis.com/vrouves-sin...,Βότανα Βρούβες-Σινάπι και διατροφική αξία~Τ...,"['καρδιά', 'κυκλοφορικό']","['Αγγελικής', 'καρδιά', 'μέλι', 'μαρούλι', 'πρ...","['καρδιά', 'κυκλοφορικό']['Αγγελικής', 'καρδιά..."
665,https://www.proionta-tis-fisis.com/ydrastis-gi...,Βότανα Υδραστίς για λοιμώξεις του αναπνευστ...,"['αναπνευστικής', 'αναπνευστικού', 'αυτί', 'αυ...","['Υδραστίς', 'αγγλική']","['αναπνευστικής', 'αναπνευστικού', 'αυτί', 'αυ..."
666,https://www.proionta-tis-fisis.com/yssopos-o-f...,"Βότανα Ύσσωπος ο φαρμακευτικός, Hyssopus of...","['αναπνευστικού', 'αναπνοής', 'καρδιάς', 'νευρ...","['Άνηθος', 'Ύσσωπος', 'καρδιάς']","['αναπνευστικού', 'αναπνοής', 'καρδιάς', 'νευρ..."


In [15]:
import numpy as np
df_texts['entities'] = ''
for i in range(df_texts.shape[0]):    
    df_texts.at[i,'entities'] = df_texts.iloc[i]['organs']+df_texts.iloc[i]['herbs']

In [16]:
df_texts

,link,text,organs,herbs,entities
0,https://www.proionta-tis-fisis.com/10-apo-tis-...,Βότανα 10 από τις εκπληκτικές χρήσεις της α...,"['Έντερο', 'έντερο', 'δέρμα', 'δέρματος', 'κυκ...","['Αλόη', 'Αλόης', 'αλόη', 'αλόης', 'ελαιόλαδο'...","['Έντερο', 'έντερο', 'δέρμα', 'δέρματος', 'κυκ..."
1,https://www.proionta-tis-fisis.com/10-hriseis-...,Βότανα 10 χρήσεις για την υπέροχη λεβάνταΒό...,"['αυτιά', 'δέρμα', 'μαλλιά', 'μαλλιών', 'μυαλό...","['θυμάρι', 'λεβάντα', 'λεβάντας', 'μέντα', 'μα...","['αυτιά', 'δέρμα', 'μαλλιά', 'μαλλιών', 'μυαλό..."
2,https://www.proionta-tis-fisis.com/10-logi-gia...,Βότανα 10 λόγοι για να φυτέψεις ρίγανη στη ...,['πεπτικό'],"['Ρίγανη', 'δεντρολίβανο', 'ρίγανη', 'ρίγανης']","['πεπτικό']['Ρίγανη', 'δεντρολίβανο', 'ρίγανη'..."
3,https://www.proionta-tis-fisis.com/10-votana-p...,Βότανα 10 βότανα που μπορούν να νικούν τις ...,"['Καρυδιά', 'αναπνευστικού', 'δέρματος', 'καρυ...","['Αλόη', 'Ευκάλυπτος', 'Καρυδιά', 'αγριοκασταν...","['Καρυδιά', 'αναπνευστικού', 'δέρματος', 'καρυ..."
4,https://www.proionta-tis-fisis.com/15-logoi-na...,Βότανα 15 λόγοι να βάλεις στην ζωή σου το Γ...,"['ήπαρ', 'αίμα', 'αγγεία', 'δέρμα', 'καρδιάς',...","['Γαϊδουράγκαθο', 'γαιδουράγκαθο', 'γαϊδουράγκ...","['ήπαρ', 'αίμα', 'αγγεία', 'δέρμα', 'καρδιάς',..."
...,...,...,...,...,...
663,https://www.proionta-tis-fisis.com/vrionia-i-k...,Βότανα Βρυωνία η κρητική - ΙδιότητεςΒόταναΒ...,"['δέρμα', 'μυϊκών', 'πεπτικού']","['άμπελο', 'βρυωνία', 'κισσού', 'μανδραγόρα', ...","['δέρμα', 'μυϊκών', 'πεπτικού']['άμπελο', 'βρυ..."
664,https://www.proionta-tis-fisis.com/vrouves-sin...,Βότανα Βρούβες-Σινάπι και διατροφική αξία~Τ...,"['καρδιά', 'κυκλοφορικό']","['Αγγελικής', 'καρδιά', 'μέλι', 'μαρούλι', 'πρ...","['καρδιά', 'κυκλοφορικό']['Αγγελικής', 'καρδιά..."
665,https://www.proionta-tis-fisis.com/ydrastis-gi...,Βότανα Υδραστίς για λοιμώξεις του αναπνευστ...,"['αναπνευστικής', 'αναπνευστικού', 'αυτί', 'αυ...","['Υδραστίς', 'αγγλική']","['αναπνευστικής', 'αναπνευστικού', 'αυτί', 'αυ..."
666,https://www.proionta-tis-fisis.com/yssopos-o-f...,"Βότανα Ύσσωπος ο φαρμακευτικός, Hyssopus of...","['αναπνευστικού', 'αναπνοής', 'καρδιάς', 'νευρ...","['Άνηθος', 'Ύσσωπος', 'καρδιάς']","['αναπνευστικού', 'αναπνοής', 'καρδιάς', 'νευρ..."


In [17]:
df_texts.to_csv('../data/proionta_tis_fisis_votana_entities.csv', index=False)